! pip install transformers

In [1]:
from fastembed import TextEmbedding
from pprint import pprint
from transformers import AutoTokenizer
import pathlib, json
import re

In [11]:
model_name = 'snowflake/snowflake-arctic-embed-m-long'
emb_model = TextEmbedding(model_name=model_name)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [3]:
# Load tokenizer (must match the model)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
filename = 'bfp-a3447q.pdf"
input_path = "data/" + filename
output_path= input_path.split('.')[0]+'.txt'
image_path = "data/images"

In [7]:
for index, chapter in enumerate(data):
    print(f'{index}) {chapter[1]}')

0) 1 General configuration
1) 1.1 Structural equipment
2) 1.1.1 Standard structural equipment
3) 1.1.2 Special specifications
4) 1.1.3 Options
5) 1.1.4 Maintenance parts
6) 1.2 Model type name of robot
7) 1.2.1 How to identify the robot model
8) 1.2.2 Combination of the robot arm and the controller
9) 1.3 CE marking specifications
10) 1.4 Indirect export
11) 1.5 Instruction manuals
12) 1.6 Contents of the structural equipment
13) 1.6.1 Robot arm
14) 1.6.2 Controller
15) 1.7 Contents of the Option equipment and special specification
16) 2 Robot arm
17) 2.1 Standard specifications
18) 2.1.1 Basic specifications
19) (1) RH-3CH-Sxx
20) (2) RH-6CH-Sxx
21) 2.1.2 The counter-force applied to the installation surface
22) 2.2 Definition of specifications
23) 2.2.1 Pose repeatability
24) 2.2.2 Mass capacity
25) 2.2.3 Relationships Among Mass Capacity, Speed, and Acceleration/Deceleration Speed
26) (1) Setting Load Capacity and Size (Hand Conditions)
27) 2.2.4 Vibrations at the Tip of the Arm dur

In [5]:
def chunk_text_by_lines(text, tokenizer, chunk_size):
    """
    Splits text into chunks based on token count, ensuring cuts only happen
    between lines (preserving \n).
    """
    lines = text.splitlines(keepends=True)  # keep '\n' at end of each line
    chunks, current_chunk, current_tokens = [], [], 0
    
    for line in lines:
        line_tokens = tokenizer.encode(line, add_special_tokens=False)
        if current_tokens + len(line_tokens) > chunk_size:
            # flush current chunk
            chunks.append("".join(current_chunk).strip())
            current_chunk, current_tokens = [], 0
        current_chunk.append(line)
        current_tokens += len(line_tokens)
    
    # add last chunk
    if current_chunk:
        chunks.append("".join(current_chunk).strip())
    
    return chunks

In [6]:
json_read = pathlib.Path(output_path).read_text()
data = json.loads(json_read)

In [7]:
max_tokens = 0
chunks = []
token_limit = round(2048*0.9)
print(f'Token threshold: {token_limit}')
for chunk in data:
    text = chunk[-1]
    # Replace <br> with semicolons or newlines
    text = re.sub(r"<br\s*/?>", "; ", text)
    
    # Strip off excessive whitespace
    text = re.sub(r" +", " ", text)
    text = re.sub(r"~", "", text)
    chunk[-1] = chunk_text_by_lines(text, tokenizer, token_limit)
    chunks.extend(chunk[-1])
    if len(chunk[-1]) > 1:
        print(f"Text length: {len(text)}\tChunks count: {len(chunk[-1])}\t for {chunk[1]}")


Token threshold: 1843
Text length: 4782	Chunks count: 2	 for 1.2.2 Combination of the robot arm and the controller
Text length: 5014	Chunks count: 2	 for 1.3 CE marking specifications
Text length: 5162	Chunks count: 2	 for (2) RH-6CH-Sxx
Text length: 4488	Chunks count: 2	 for (1) Setting Load Capacity and Size (Hand Conditions)
Text length: 3917	Chunks count: 2	 for (1) Normal environmental specification
Text length: 7392	Chunks count: 2	 for 3.5 Dedicated input/output
Text length: 5025	Chunks count: 2	 for (1) Teaching pendant (T/B)
Text length: 11510	Chunks count: 3	 for (2) Parallel I/O interface
Text length: 13749	Chunks count: 3	 for (4) Parallel I/O unit
Text length: 7526	Chunks count: 2	 for (6) CC-Link interface
Text length: 7519	Chunks count: 2	 for 4.1.2 Descriptions of changed functions/specifications
Text length: 8881	Chunks count: 2	 for 4.2 List of commands


In [77]:
print(*data[86][-1], sep='\n---Chunk Split---\n')

(4) Parallel I/O unit


- Order type: 2A-RZ361 (Sink type)
2A-RZ371 (Source type)


- Outline

This is used to expand the external inputs and outputs. One one equal with this unit
is built into the control unit among controllers the standard.
The connection cable is not included. .Prepare the optional external input/output
cable (2A-CBL05 or 2A-CBL15).

Use 2A-RZ361 if the external input/output signal logic is of the sink type and 2ARZ371 for source type signal logic.

![](data/images/bfp-a3447q.pdf-94-0.png)


Notes) Although the combined use with the parallel I/O interface (2D-TZ368) of another option is also possible,

 please use the setup of the station number by the different number separately. The station number is auto
matically fixed by the position of the option slot which installed the parallel I/O interface in 0-1.


- Configuration
Table 3-20 ：Configuration device

|Part name|Type|Qty.|Mass (kg) *Note 1)|Remarks|
|---|---|---|---|---|
|Parallel I/O unit|2A-RZ361|Either one

In [8]:
from qdrant_client import QdrantClient, models

In [16]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [18]:
# Define the collection name
collection_name = "bfp-a3447q"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=emb_model.embedding_size,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [17]:
client.collection_exists('bfp-a3447q')

False

In [19]:
data[:3]

[[1, '1 General configuration', 14, ['1 General configuration']],
 [2,
  '1.1 Structural equipment',
  14,
  ['1.1 Structural equipment\n\nStructural equipment consists of the following types.']],
 [3,
  '1.1.1 Standard structural equipment',
  14,
  ['1.1.1 Standard structural equipment\n\nThe following items are enclosed as a standard.\n(1) Robot arm\n(2) Controller\n(3) Machine cable\n(4) Robot arm installation bolts\n(5) Safety manual, CD-ROM (Instruction manual)\n(6) Guarantee card']]]

In [22]:
points = []
parent_chapter = ""
id = 0
title = 'RH-3CH-Sxx/RH-6CH-Sxx Special Specifications Manual' # can be obtained from doc metadata
for chapter in data:
    # elements of data list:
        # 0 - chapter level
        # 1 - chapter name
        # 2 - page number (1-based)
        # 3 - list of chunks of text
    if chapter[0] == 1:
        root_chapter = chapter[1]
    for chunk in chapter[-1]:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=chunk, model=model_name),
            payload={
                "content": chunk,
                "main_chapter": root_chapter,
                "chapter": chapter[1],
                "manual": title,
                "page": chapter[2]
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [23]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)